## Prac preavious session in building a model and training 

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim


torch.manual_seed(20)

class PracANN(nn.Module):
    def __init__(self):
        super(PracANN, self).__init__()
        self.fc1 = nn.Linear(256*256*3, 1084)
        self.fc2 = nn.Linear(1084, 512)
        self.fc3 = nn.Linear(512, 257)

    def forward(self, x):
        x = x.view(-1, 256*256*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def get_accuracy(model, train = False, data_loader=None):
    if train:
        model.train()
    else:
        model.eval()
    
    correct = 0
    total = 0

    for imgs, labels in data_loader:
        outputs = model(imgs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return correct / total

def train(model, train_loader, valid_loader, lr=0.01, num_epochs = 1):
    
    criterian = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    iters, losses, train_acc, val_acc = [], [], [], []

    iteration_count = 0

    for epoch in range(num_epochs):
        for imgs, labels in train_loader:
            model.train()
            out = model(imgs)
            loss = criterian(out, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # save the current training information
            iters.append(iteration_count)
            losses.append(loss.item())
            train_acc.append(get_accuracy(model, train=True, data_loader=train_loader))
            val_acc.append(get_accuracy(model, train=False, data_loader=valid_loader))
            iteration_count += 1
        print("Epoch: ", epoch+1, " Loss: ", loss.item(), " Train Accuracy: ", train_acc[-1], " Validation Accuracy: ", val_acc[-1])

    # plotting 
    plt.title("Training Curve")        
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: ", train_acc[-1])
    print("Final Validation Accuracy: ", val_acc[-1])
    
    return iters, losses, train_acc, val_acc

model = PracANN()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [23]:
# Dataset preparation 

from torchvision import datasets, transforms 
from skimage import color
import numpy as np

my_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(),
    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.ToTensor(),
    
])

Caltech_data = datasets.Caltech256('data', download=True,
                             transform=my_transform)

print("Total number of images: ", len(Caltech_data))

Total number of images:  30607


In [24]:
# split into training and validation set

train_size = int(0.8 * len(Caltech_data))
valid_size = len(Caltech_data) - train_size

train_dataset, valid_dataset = torch.utils.data.random_split(Caltech_data, [train_size, valid_size])

# create dataloaders 

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [25]:
## sanity check the data
## need to add to cuda if you want to train on gpu

model = PracANN()
train(model, train_loader, valid_loader, num_epochs=20)

correct = 0
total = 0

for imgs, labels in valid_loader:
    outputs = model(imgs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print("Test Accuracy: ", correct / total)

KeyboardInterrupt: 

## Transfer Learning

In [4]:
import torchvision.models

alexNet = torchvision.models.alexnet(pretrained=True)

c:\Users\admin\miniforge3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\admin\miniforge3\envs\ml\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
alexNet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
alexNet.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [7]:
import matplotlib.pyplot as plt
import numpy as np

# load colour image 

img = plt.imread()

TypeError: imread() missing 1 required positional argument: 'fname'

In [ ]:
alexNetConv = alexNet.features[0]
y = alexNetConv(x)